In [5]:
# First we have to install selenium
# !pip install selenium
# geckodriver is at /home/toorajtaraz/Downloads/geckodriver-v0.33.0-linux64/
path_to_geckodriver = '/home/toorajtaraz/Downloads/New_Folder/geckodriver'

In [7]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Create a new instance of the Firefox driver
driver = webdriver.Firefox(executable_path=path_to_geckodriver)

# go to the google home page
driver.get("http://www.google.com")


In [8]:
# We want to implement an scraper for google scholar
# We look for a researcher or subject and then we want to get all the abstracts of their papers

# We first go to the google scholar page
driver.get("https://scholar.google.com/")

In [9]:
keyword_to_be_searched = 'Validation of regression models: methods and examples'
scholar_uri = 'https://scholar.google.com/'
driver.get("https://scholar.google.com/")
# We find the search box and enter the keyword
search_bar_id = 'gs_hdr_tsi'
search_bar = driver.find_element_by_id(search_bar_id)
search_bar.send_keys(keyword_to_be_searched)
search_bar.send_keys(Keys.RETURN)

In [10]:
# Now we want to get first author with link
# all the autors are inside an a tag which itself is inside a div tag with class gs_a
# we want to loop over all the div tags with class gs_a
# and then get the first a tag inside each div tag
first_author_link = ''
for div_tag in driver.find_elements_by_class_name('gs_a'):
    try:
        # we get the first a tag inside the div tag
        first_a_tag = div_tag.find_element_by_tag_name('a')
        # we get the text of the first a tag
        first_author = first_a_tag.text
        # we get the link of the first a tag
        first_author_link = first_a_tag.get_attribute('href')
        print(first_author)
        print(first_author_link)
        break
    except Exception:
        continue

RD Snee
https://scholar.google.com/citations?user=SelQvxgAAAAJ&hl=pl&oi=sra


In [11]:
if first_author_link != '':
    driver.get(first_author_link)

In [14]:
# now we have to press button with id gsc_bpf_more until we get all the papers
# we have to press the button until we get the button with id gsc_bpf_more disabled
# we want to import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
while True:
    # try:
        # check if button is disabled
    more_button = driver.find_element_by_id('gsc_bpf_more')
    more_button.click()
    # now we wait for loading to be done
    # we wait for the button with id gsc_bpf_more to be enabled
    try:
        WebDriverWait(driver, 10).until(
            lambda x: x.find_element_by_id('gsc_bpf_more').is_enabled()
        )
    except Exception:
        break
    # except Exception:
    #     break

In [15]:
researchers_university = ''
researchers_research_interests = ''
researchers_papers = []
# we want to import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
# path to all links to papers : /html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr[X]/td[1]/a
paper_count = driver.find_element_by_id('gsc_lwp').text
paper_count = paper_count.split('–')[1].split('SHOW MORE')[0]
print(paper_count)
paper_count = int(paper_count)

i = 0
for i in range(1, paper_count + 1):
    try:
        xpath_to_paper = '/html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr[' + str(i) + ']/td[1]/a'
        href = driver.find_element_by_xpath(xpath_to_paper).get_attribute('href')
        # Open new tab
        # driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't')
        # ActionChains(driver).key_down(Keys.CONTROL).send_keys('t').key_up(Keys.CONTROL).perform()
        driver.execute_script('''window.open("http://bings.com","_blank");''')
        # Switch to the new window which is second window
        driver.switch_to.window(driver.window_handles[1])
        # Open the link in new tab
        driver.get(href)
        # get the title of the paper
        try:
            paper_title = driver.find_element_by_id('gsc_oci_title').text
        except Exception as e:
            print('couldn\'t get paper title')
            print(e)
            paper_title = ''
        try:
            publication_date = driver.find_element_by_xpath('/html/body/div/div[7]/div[2]/div/div[2]/div[2]/div[2]').text
        except Exception as e:
            print('couldn\'t get publication date')
            print(e)
            publication_date = ''
        try:
            cited = driver.find_element_by_xpath('/html/body/div/div[7]/div[2]/div/div[2]/div[9]/div[2]/div[1]/a').text
        except Exception as e:
            print('couldn\'t get cited')
            print(e)
            cited = ''
        try:
            description = driver.find_element_by_id('gsc_oci_descr').text 
        except Exception as e:
            print('couldn\'t get description')
            print(e)
            description = ''
        print(paper_title)
        print(publication_date)
        print(cited)
        print(description)
        import time
        time.sleep(1)
        # close the tab
        driver.close()
        # Switch back to the first tab
        driver.switch_to.window(driver.window_handles[0])
        # add the paper to the list of papers
        researchers_papers.append({
            'title': paper_title,
            'publication_date': publication_date,
            'cited': cited,
            'description': description
        })
    except Exception as e:
        print(e)
        continue



338
Validation of regression models: methods and examples
1977/11/1
Cited by 1458
Methods to determine the validity of regression models include comparison of model predictions and coefficients with theory, collection of new data to check model predictions. comparison of results with theoretical model calculations, and data splitting or cross-validation in which a portion of the data is used to estimate the model coefficients, and the remainder of the data is used to measure the prediction accuracy of the model. An expository review of these methods is presented. It is concluded that data splitting is an effective method of model validation when it is not practical to collect new data to test the model. The DUPLEX algorithm, developed by R. W. Kennard, is recommended for dividing the data into the estimation set and prediction set when there is no obvious variable such as time to use as a basis to split the data. Several examples are included to illustrate the various methods of model 

In [21]:
driver.close()
        # Switch back to the first tab
driver.switch_to.window(driver.window_handles[0])

In [16]:
# now we save the data in a csv file
import csv
with open('selenium_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['title', 'publication_date', 'cited', 'description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for paper in researchers_papers:
        writer.writerow(paper)